In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from matplotlib.image import imread
import numpy as np
%matplotlib inline


In [30]:
import os
import json
from skimage.color import rgb2gray
from skimage.transform import resize

import random

# Get Image Data

In [4]:
image_data_directory = "Dataset/Color"
annotation_url = 'Dataset/annotation.json'

In [5]:
with open(annotation_url) as annotation:
    annotation_data = json.load(annotation)

In [6]:
colors = [
		[100.,  100.,  100.], 
		[100.,    0.,    0.],
		[150.,    0.,    0.],
		[200.,    0.,    0.],
		[255.,    0.,    0.],
		[100.,  100.,    0.],
		[150.,  150.,    0.],
		[200.,  200.,    0.],
		[255.,  255.,    0.],
		[  0.,  100.,   50.],
		[  0.,  150.,   75.],
		[  0.,  200.,  100.],
		[  0.,  255.,  125.],
		[  0.,   50.,  100.],
		[  0.,   75.,  150.],
		[  0.,  100.,  200.],
		[  0.,  125.,  255.],
		[100.,    0.,  100.],
		[150.,    0.,  150.],
        [200.,    0.,  200.],
        [255.,    0.,  255.]]

colors = np.divide(np.array(colors), 255)
print(colors)

[[0.39215686 0.39215686 0.39215686]
 [0.39215686 0.         0.        ]
 [0.58823529 0.         0.        ]
 [0.78431373 0.         0.        ]
 [1.         0.         0.        ]
 [0.39215686 0.39215686 0.        ]
 [0.58823529 0.58823529 0.        ]
 [0.78431373 0.78431373 0.        ]
 [1.         1.         0.        ]
 [0.         0.39215686 0.19607843]
 [0.         0.58823529 0.29411765]
 [0.         0.78431373 0.39215686]
 [0.         1.         0.49019608]
 [0.         0.19607843 0.39215686]
 [0.         0.29411765 0.58823529]
 [0.         0.39215686 0.78431373]
 [0.         0.49019608 1.        ]
 [0.39215686 0.         0.39215686]
 [0.58823529 0.         0.58823529]
 [0.78431373 0.         0.78431373]
 [1.         0.         1.        ]]


In [213]:
def getImageData(image, pos, resize_ratio_x, resize_ratio_y, name):
    
    newpos_x = pos[:,0] / resize_ratio_x
    newpos_y = pos[:,1] / resize_ratio_y
    
    top_left_x = np.amin(newpos_x) - 2
    top_left_y = np.amin(newpos_y) - 2
    
    width = np.amax(newpos_x) - top_left_x + 2
    height = np.amax(newpos_y) - top_left_y + 2

    return top_left_x, top_left_y, width, height

In [214]:
def getCropImageData(image, pos,  name):
    
    top_left_x = np.amin(pos[:,0]) - 10
    top_left_y = np.amin(pos[:,1]) - 10
    
    width = np.amax(pos[:,0]) - top_left_x + 20
    height = np.amax(pos[:,1]) - top_left_y + 20


    return int(top_left_x), int(top_left_y), int(width), int(height)

In [215]:
def plotImageData(image, annotation_data, resize_ratio_x, resize_ratio_y, name):
    plt.figure(figsize=(20,10))
#     image = rgb2gray(image)
    plt.imshow(image)
    pos = np.array(annotation_data[name])
    
    pos[:,0] /= resize_ratio_x
    pos[:,1] /= resize_ratio_y
    
    top_left_x = np.amin(pos[:,0]) - 2
    top_left_y = np.amin(pos[:,1]) - 2
    
    width = np.amax(pos[:,0]) - top_left_x + 2
    height = np.amax(pos[:,1]) - top_left_y + 2

    
    rect = patches.Rectangle((top_left_x,top_left_y),width,height,linewidth=1,edgecolor='r',facecolor='none')
    
    plt.gca().add_patch(rect)
    
    plt.scatter(x=pos[:, 0], y=pos[:, 1], c=colors, s=10, alpha=0.7)
    plt.show()
    

In [216]:
def load_part_data(names, data_directory, annotation_data):

    reshaped_x = 180
    reshaped_y = 320
    cropped_resize_x = 130
    cropped_resize_y = 130
    
    allImages = np.empty((len(names), reshaped_x, reshaped_y, 3))
    allCroppedImages = np.empty((len(names), cropped_resize_x, cropped_resize_y, 3))
    allBoxs = np.empty((len(names), 4))
    allLabels = []
    
    for i in range(len(names)):
        name = names[i]
        label = 0 if name[-1] == 'L' else 1
            
        image = imread(data_directory + "/" + name[:-2] + ".jpg")
        pos = np.array(annotation_data[name])

        resize_ratio_x = image.shape[0] / reshaped_x
        resize_ratio_y = image.shape[1] / reshaped_y
        resized_image = resize(image, (reshaped_x, reshaped_y, 3))
        
        top_x, top_y, width, height = getImageData(image, pos, 
                                                          resize_ratio_x, resize_ratio_y, name)        
        boundingBox = np.array([top_x, top_y, width, height])
        
        newLabel = np.array([label])
        
        allImages[i] = resized_image
        allBoxs[i] = boundingBox
        
        allLabels.append(newLabel) 
        
        
        
        top_x, top_y, width, height = getCropImageData(image, pos, name)    
        
        cropped_image = image[top_y: top_y + height, top_x: top_x + width, :]
        
        cropped_image = resize(cropped_image, (cropped_resize_x, cropped_resize_y, 3))
        allCroppedImages[i] = cropped_image
        
    
    print(allImages.shape)
    print(allBoxs.shape)
    
    return allImages, allCroppedImages, np.concatenate(allLabels), allBoxs
        


In [217]:
def load_data(data_directory, annotation_data):
    names = list(annotation_data.keys())

    print("Get train")
    train_data = np.array(names[:1000])
    X_train, X_crop_train, Y_train_label, Y_train_box = load_part_data(train_data, data_directory, annotation_data)
    
    
    print("Get valid")
    valid_data = np.array(names[2000:2500])
    X_val, X_crop_val, Y_val_label, Y_val_box = load_part_data(valid_data, data_directory, annotation_data)
    
    print("Get test")
    test_data = np.array(names[3000:3500])
    X_test, X_crop_test, Y_test_label, Y_test_box = load_part_data(test_data, data_directory, annotation_data)
    
    mean_image = np.mean(X_train, axis=0)

    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    
    """
    for name in names:
        if count > 10:
            break
        image = imread(data_directory + "/" + name[:-2] + ".jpg")
        label, top_x, top_y, width, height = getImageData(image, annotation_data, name)
        plotImageData(image, annotation_data, name)
        count += 1    
    """
    
    return (X_train, X_crop_train, Y_train_label, Y_train_box, 
            X_val, X_crop_val, Y_val_label, Y_val_box, 
            X_test, X_crop_test, Y_test_label, Y_test_box)

            

In [218]:
(X_train, X_crop_train, Y_train_label, Y_train_box, 
 X_val, X_crop_val, Y_val_label, Y_val_box, 
 X_test, X_crop_test, Y_test_label, Y_test_box) = load_data(image_data_directory, annotation_data)

Get train


C:\Users\Wei Han Chen\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(1000, 180, 320, 3)
(1000, 4)
Get valid
(500, 180, 320, 3)
(500, 4)
Get test
(500, 180, 320, 3)
(500, 4)


In [219]:
print(X_train.shape)
print(X_crop_train.shape)
print(Y_train_label.shape)
print(Y_train_box.shape)

print(X_val.shape)
print(X_crop_val.shape)
print(Y_val_label.shape)
print(Y_val_box.shape)

print(X_test.shape)
print(X_crop_test.shape)
print(Y_test_label.shape)
print(Y_test_box.shape)

(1000, 180, 320, 3)
(1000, 130, 130, 3)
(1000,)
(1000, 4)
(500, 180, 320, 3)
(500, 130, 130, 3)
(500,)
(500, 4)
(500, 180, 320, 3)
(500, 130, 130, 3)
(500,)
(500, 4)


# Image Model

image data will have size N x W x H

N = number of images
W = width
H = height

notice that we dont have any a new dimension C that contains RGB values because RGB values
wont affect much in this case.



In [14]:
# remove old variables

tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 1080, 1920])
Y_label = tf.placeholder(tf.int32, [None])
Y_box_pos = tf.placeholder(tf.int32, [None, 4])
is_training = tf.placeholder(tf.bool)


In [221]:
tf.keras.backend.image_data_format()

'channels_last'